### 목표
- selenium 라이브러리를 통해서 네이버 지도 맛집 데이터를 수집해보기

In [27]:
# 라이브러리 불러오기
  
try :
    import os  # 운영체제 관련 작업 (폴더 생성, 삭제 등)
except ModuleNotFoundError:
    !pip install os
    
try :
    # 페이지 로딩 시간을 부여하는 라이브러리
    import time  # 시간 관련 기능 (대기, 지연)
except ModuleNotFoundError:
    !pip install time 
    
try:
    # 컴퓨터용 브라우저
    # 컴퓨터용 키보드
    # 선택지 지정방식
    from selenium import webdriver as wb
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
except ModuleNotFoundError:
    !pip install selenium
    
try : 
    # 반복문 진행 상황을 확인할 수 있는 도구
    from tqdm import tqdm
except ModuleNotFoundError :
    !pip install tqdm

try :
    # 랭과 열로 이루어진 표와 같은 형태를 다룰 수 있는 라이브러리
    import pandas as pd 
except ModuleNotFoundError :
    !pip install pandas

print(f'필요한 라이브러리 import 완료')

필요한 라이브러리 import 완료


In [28]:
# 정보

URL = 'https://map.naver.com/p?c=15.00,0,0,0,dh'

# 메인폴더 위치
ORIGIN_URL = "C:\\Users\\smhrd1\\Documents\\GitHub\\Crawling\\Study"

# 지도의 검색창 위치
MAP_IMPUT = 'input.input_search'

# 지도에서 식당의 이름
MAP_NAME = 'TYaxT'

# 지도에서 식당의 주소들
MAP_ADDRESS_1 = 'LDgIH'
MAP_ADDRESS_2 = 'PkgBl'

# 지도에서 end를 위한 위치 선택
MAP_SCROLL = 'Ryr1F'

# 지도에서  다음 페이지의 버튼
NEXT_BUTTON = 'a.mBN2s + a.eUTV2 > svg.yUtES' 

In [29]:
driver = wb.Chrome()
driver.get(URL) 
time.sleep(3)

In [30]:
map_input = driver.find_element(By.CSS_SELECTOR, MAP_IMPUT)  
for i in range(10) :
    map_input.send_keys(Keys.BACK_SPACE)
map_input.send_keys('순천 맛집')
map_input.send_keys(Keys.ENTER)

time.sleep(2)   

In [31]:
origin_get_All_List = driver.find_elements(By.CSS_SELECTOR, MAP_NAME)

In [32]:
# iframe 창 전환하는 함수

driver.switch_to.frame('searchIframe') # id 값만 

In [33]:
get_All_List = []
get_All_name_List = []

def find_name():
    get_All_name_List.clear()
    get_All_List.clear()

    while True:
        map_scr = driver.find_element(By.CLASS_NAME, MAP_SCROLL)
        origin_len = len(driver.find_elements(By.CSS_SELECTOR, 'li'))

        for _ in range(2):  # 두 번 스크롤 내림
            map_scr.send_keys(Keys.END)
            time.sleep(0.5)

        later_len = len(driver.find_elements(By.CSS_SELECTOR, 'li'))

        if origin_len == later_len:  # 스크롤 후 변화가 없으면 마지막 페이지
            print("페이지의 끝인지 검사 중")
            map_scr.send_keys(Keys.END)
            time.sleep(1)
            check_len = len(driver.find_elements(By.CSS_SELECTOR, 'li'))

            if later_len == check_len:
                print("페이지가 끝났습니다.")
                break
            else:
                print("페이지가 끝나지 않아서 다시 스크롤을 시작합니다.")

    time.sleep(2) 
    get_All_List.extend(driver.find_elements(By.CLASS_NAME, MAP_NAME))
 
    get_All_name_List.extend([element.text for element in get_All_List])
 


In [34]:
store_info = [] 
# 첫 번째 가게를 클릭   
store_info = [] 
# 첫 번째 가게를 클릭   
def find_address():
    store_info.clear()
    for i in tqdm(range(len(get_All_List))):
        while True:  # 무한 루프를 돌려서 실패 시 재시도
            try:
                get_All_List[i].click()
                time.sleep(0.5)
                
                driver.switch_to.default_content()
                time.sleep(0.3)
                driver.switch_to.frame('entryIframe') 
                try :
                    check = driver.find_element(By.CLASS_NAME, MAP_ADDRESS_1) 
                except :
                    check = driver.find_element(By.CLASS_NAME, MAP_ADDRESS_2) 
                store_info.append(check.text)
                
                driver.switch_to.default_content()
                time.sleep(0.3)
                driver.switch_to.frame('searchIframe')
                
                time.sleep(0.8)
                break  # 클릭 성공하면 루프 종료
            except Exception as e:
                driver.switch_to.default_content()
                driver.switch_to.frame('searchIframe')
                print(f"Error occurred: {e}, 1초 후 재시도: {i}")
                time.sleep(1)  # 실패 시 1초 후 재시도
                continue  # 실패 시 다시 시도

In [35]:
address_list = [i for i in store_info] 
address_list

[]

In [36]:
def next_page() :
    try :
        origin_page_num = driver.find_element(By.CLASS_NAME, 'mBN2s.qxokY')
        find_the_button = driver.find_element(By.CSS_SELECTOR, NEXT_BUTTON) 
        find_the_button.click()
        time.sleep(2) 
        later_page_num = driver.find_element(By.CLASS_NAME, 'mBN2s.qxokY')
        if(origin_page_num == later_page_num) :
            return False
        else :
            return True
    except :
        return False  

In [ ]:
result_name = []
result_address = []

while(True) :
    find_name() 
    find_address()

    for i in get_All_List :
        result_name.append(i)
    for i in store_info :
         result_address.append(i)

    page_check = next_page()
    if not page_check :
        break   

페이지의 끝인지 검사 중
페이지가 끝났습니다.


100%|██████████████████████████████████████████████████████████████████████████████████| 76/76 [02:43<00:00,  2.16s/it]


페이지의 끝인지 검사 중
페이지가 끝났습니다.


100%|██████████████████████████████████████████████████████████████████████████████████| 76/76 [02:47<00:00,  2.20s/it]


페이지의 끝인지 검사 중
페이지가 끝났습니다.


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [02:36<00:00,  2.24s/it]


페이지의 끝인지 검사 중
페이지가 끝났습니다.


 67%|███████████████████████████████████████████████████████                           | 47/70 [01:46<00:53,  2.33s/it]

In [ ]:
print("식당 수 : ", len(result_name))
print("주소 수 : ", len(result_address))

In [ ]:
df = pd.DataFrame({'식당이름' : result_name, '식당주소' : result_address}) 
df = df.reset_index(drop=True)  # 인덱스를 0부터 다시 설정
df.index = df.index + 1  # 인덱스를 1부터 시작하도록 변경
df

In [ ]:
df.to_csv('순천시_맛집_목록.csv', encoding = 'utf-8')